In [1]:
import os
from tqdm import tqdm

import utils
import config
import datasets
import albumentations as A
from albumentations.pytorch import ToTensorV2
from torch.utils.data import DataLoader
import validate

import numpy as np
import torch
import torchmetrics

import cv2
import matplotlib.pyplot as plt

In [2]:
import onnx
import onnxruntime

# Load Models

### No Compression

In [3]:
model_no = onnx.load('./models/onnx/no_comp_model.onnx')
onnx.checker.check_model(model_no)

### Low Compression

In [4]:
model_low = onnx.load('./models/onnx/low_comp_model.onnx')
onnx.checker.check_model(model_low)

### Medium Compression

In [5]:
model_med = onnx.load('./models/onnx/med_comp_model.onnx')
onnx.checker.check_model(model_med)

### High Compression

In [6]:
model_high = onnx.load('./models/onnx/high_comp_model.onnx')
onnx.checker.check_model(model_high)

### FASDD No Compression

In [7]:
model_fasdd_no_comp = onnx.load('./models/onnx/fasdd_no_comp.onnx')
onnx.checker.check_model(model_fasdd_no_comp)

### FASDD Medium Compression

In [8]:
model_fasdd_med = onnx.load('./models/onnx/fasdd_med.onnx')
onnx.checker.check_model(model_fasdd_med)

In [9]:
model_fasdd_med_f1mean = onnx.load('./models/onnx/fasdd_med_best_f1_mean_epoch=37.onnx')
onnx.checker.check_model(model_fasdd_med_f1mean)

### Original FP32 Model Trained with FASDD

In [10]:
model_fp32 = onnx.load('./models/onnx/BED_classifier__fused__dfire_fasdd.onnx')
onnx.checker.check_model(model_fp32)

# Datasets

In [11]:
dfire_loader = datasets.get_dfire_val_loader(shuffle=False)
fasdd_uav_loader = datasets.get_fasdd_uav_val_loader(shuffle=False)
fasdd_cv_loader = datasets.get_fasdd_cv_val_loader(shuffle=False)
val_loader = datasets.get_val_loader(shuffle=False)


TEST DFire dataset
DFire Removed wrong images: 0
DFire empty images: 2005
DFire only smoke images: 1186
DFire only fire images: 220
DFire smoke and fire images: 895

Test dataset len: 4306

TEST FASDD UAV dataset
DFire Removed wrong images: 0
DFire empty images: 1997
DFire only smoke images: 846
DFire only fire images: 35
DFire smoke and fire images: 1303

Test FASDD UAV dataset len: 4181

TEST FASDD CV dataset
DFire Removed wrong images: 0
DFire empty images: 6533
DFire only smoke images: 3902
DFire only fire images: 2091
DFire smoke and fire images: 3358

Test FASDD CV dataset len: 15884

TEST DFire dataset
DFire Removed wrong images: 0
DFire empty images: 2005
DFire only smoke images: 1186
DFire only fire images: 220
DFire smoke and fire images: 895

Test dataset len: 4306

TEST FASDD UAV dataset
DFire Removed wrong images: 0
DFire empty images: 1997
DFire only smoke images: 846
DFire only fire images: 35
DFire smoke and fire images: 1303

Test FASDD UAV dataset len: 4181

TEST FAS

# Evaluate ONNX with F1 Mean, Smoke and Fire

In [12]:
precision_metric = torchmetrics.classification.MultilabelPrecision(num_labels = config.N_CLASSES, 
                                                                   threshold = 0.5, 
                                                                   average = None).to('cpu')
recall_metric = torchmetrics.classification.MultilabelRecall(num_labels = config.N_CLASSES, 
                                                             threshold = 0.5, 
                                                             average = None).to('cpu')
accuracy_metric = torchmetrics.classification.MultilabelAccuracy(num_labels = config.N_CLASSES, 
                                                                 threshold = 0.5, 
                                                                 average = None).to('cpu')
f1_metric = torchmetrics.classification.MultilabelF1Score(num_labels = config.N_CLASSES, 
                                                          threshold = 0.5, 
                                                          average = None).to('cpu')

f1_metric_mean = torchmetrics.classification.MultilabelF1Score(num_labels = config.N_CLASSES, 
                                                               threshold = 0.5, 
                                                               average = 'macro').to('cpu')

In [13]:
def to_numpy(tensor):
    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()

In [14]:
def eval_onnx(loader, model_name):

    ort_session = onnxruntime.InferenceSession(model_name, providers=["CPUExecutionProvider"])

    precision_metric.reset()
    recall_metric.reset()
    accuracy_metric.reset()
    f1_metric.reset()
    f1_metric_mean.reset()
    
    loop = tqdm(loader, desc='Validating', leave=True)

    for batch_idx, (img, label) in enumerate(loop):

        for idx in range(config.BATCH_SIZE):
            
            ort_inputs = {ort_session.get_inputs()[0].name: to_numpy(img[idx].unsqueeze(dim=0))}
            yhat = ort_session.run(None, ort_inputs)
            yhat = np.array(yhat)
            #yhat = torch.tensor(yhat).squeeze(dim=0)
            yhat = torch.sigmoid(torch.tensor(yhat).squeeze(dim=0))
            target = label[idx].unsqueeze(dim=0)

            precision_metric.update(yhat, target)
            recall_metric.update(yhat, target)
            accuracy_metric.update(yhat, target)
            f1_metric.update(yhat, target)
            f1_metric_mean.update(yhat, target)
    
    precision = precision_metric.compute()
    recall = recall_metric.compute()
    accuracy = accuracy_metric.compute()
    f1 = f1_metric.compute()
    f1_mean = f1_metric_mean.compute()

    precision_metric.reset()
    recall_metric.reset()
    accuracy_metric.reset()
    f1_metric.reset()
    f1_metric_mean.reset()

    print(f'SMOKE -> Precision: {precision[0]:.4f} - Recall: {recall[0]:.4f} - Accuracy: {accuracy[0]:.4f} - F1: {f1[0]:.4f}')
    print(f'FIRE -> Precision: {precision[1]:.4f} - Recall: {recall[1]:.4f} - Accuracy: {accuracy[1]:.4f} - F1: {f1[1]:.4f}')
    print(f'Mean F1 Score: {f1_mean.item():.4f}')
    
    return (
        {
        'Accuracy': [accuracy[0].item(), accuracy[1].item()],
        'Precision': [precision[0].item(), precision[1].item()],
        'Recall': [recall[0].item(), recall[1].item()],
        'F1': [f1[0].item(), f1[1].item()],
        'F1 mean': f1_mean.item(),
        }
    )

# Models Trained with DFire Evaluated with FASDD UAV, CV and DFire dataset

In [15]:
print("__________________________________ FASDD NO COMPRESSION  __________________________________")
_ = eval_onnx(val_loader, './models/onnx/no_comp_model.onnx')

print("\n________________________________ FASDD LOW COMPRESSION __________________________________")
_ = eval_onnx(val_loader, './models/onnx/low_comp_model.onnx')

print("\n________________________________ FASDD MEDIUM COMPRESSION _______________________________")
_ = eval_onnx(val_loader, './models/onnx/med_comp_model.onnx')

print("\n________________________________ FASDD HIGH COMPRESSION _________________________________")
_ = eval_onnx(val_loader, './models/onnx/high_comp_model.onnx')

__________________________________ FASDD NO COMPRESSION  __________________________________


Validating: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 380/380 [01:18<00:00,  4.84it/s]


SMOKE -> Precision: 0.5922 - Recall: 0.6984 - Accuracy: 0.6311 - F1: 0.6409
FIRE -> Precision: 0.7772 - Recall: 0.8044 - Accuracy: 0.8618 - F1: 0.7906
Mean F1 Score: 0.7157

________________________________ FASDD LOW COMPRESSION __________________________________


Validating: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 380/380 [01:07<00:00,  5.60it/s]


SMOKE -> Precision: 0.5987 - Recall: 0.7312 - Accuracy: 0.6423 - F1: 0.6584
FIRE -> Precision: 0.7499 - Recall: 0.8546 - Accuracy: 0.8605 - F1: 0.7989
Mean F1 Score: 0.7286

________________________________ FASDD MEDIUM COMPRESSION _______________________________


Validating: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 380/380 [01:06<00:00,  5.73it/s]


SMOKE -> Precision: 0.6278 - Recall: 0.7426 - Accuracy: 0.6711 - F1: 0.6804
FIRE -> Precision: 0.7985 - Recall: 0.8205 - Accuracy: 0.8747 - F1: 0.8094
Mean F1 Score: 0.7449

________________________________ FASDD HIGH COMPRESSION _________________________________


Validating: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 380/380 [01:05<00:00,  5.81it/s]

SMOKE -> Precision: 0.6341 - Recall: 0.7438 - Accuracy: 0.6769 - F1: 0.6846
FIRE -> Precision: 0.7909 - Recall: 0.8219 - Accuracy: 0.8718 - F1: 0.8061
Mean F1 Score: 0.7453


# Models Trained with FASDD Evaluated with FASDD UAV, CV and DFire, together and separated

### Whole Dataset

In [16]:
print("__________________________________ FASDD NO COMPRESSION __________________________________")
_ = eval_onnx(val_loader, './models/onnx/fasdd_no_comp.onnx')

print("\n________________________________ FASDD MEDIUM COMPRESSION ________________________________")
_ = eval_onnx(val_loader, './models/onnx/fasdd_med.onnx')

__________________________________ FASDD NO COMPRESSION __________________________________


Validating: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 380/380 [01:19<00:00,  4.75it/s]


SMOKE -> Precision: 0.9193 - Recall: 0.8669 - Accuracy: 0.9014 - F1: 0.8924
FIRE -> Precision: 0.8839 - Recall: 0.9701 - Accuracy: 0.9490 - F1: 0.9250
Mean F1 Score: 0.9087

________________________________ FASDD MEDIUM COMPRESSION ________________________________


Validating: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 380/380 [01:15<00:00,  5.06it/s]

SMOKE -> Precision: 0.9225 - Recall: 0.8672 - Accuracy: 0.9031 - F1: 0.8940
FIRE -> Precision: 0.9007 - Recall: 0.9717 - Accuracy: 0.9561 - F1: 0.9348
Mean F1 Score: 0.9144


### Only DFire

In [17]:
print("__________________________________ FASDD NO COMPRESSION __________________________________")
_ = eval_onnx(dfire_loader, './models/onnx/fasdd_no_comp.onnx')
print("\n________________________________ FASDD MEDIUM COMPRESSION ________________________________")
_ = eval_onnx(dfire_loader, './models/onnx/fasdd_med.onnx')

__________________________________ FASDD NO COMPRESSION __________________________________


Validating: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 67/67 [00:13<00:00,  5.07it/s]


SMOKE -> Precision: 0.9066 - Recall: 0.8982 - Accuracy: 0.9065 - F1: 0.9024
FIRE -> Precision: 0.9163 - Recall: 0.9270 - Accuracy: 0.9592 - F1: 0.9216
Mean F1 Score: 0.9120

________________________________ FASDD MEDIUM COMPRESSION ________________________________


Validating: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 67/67 [00:11<00:00,  5.60it/s]

SMOKE -> Precision: 0.9108 - Recall: 0.9011 - Accuracy: 0.9100 - F1: 0.9059
FIRE -> Precision: 0.9221 - Recall: 0.9387 - Accuracy: 0.9636 - F1: 0.9304
Mean F1 Score: 0.9182


### Only FASDD UAV

In [18]:
print("__________________________________ FASDD NO COMPRESSION __________________________________")
_ = eval_onnx(fasdd_uav_loader, './models/onnx/fasdd_no_comp.onnx')
print("\n________________________________ FASDD MEDIUM COMPRESSION ________________________________")
_ = eval_onnx(fasdd_uav_loader, './models/onnx/fasdd_med.onnx')

__________________________________ FASDD NO COMPRESSION __________________________________


Validating: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 65/65 [00:20<00:00,  3.15it/s]


SMOKE -> Precision: 0.9933 - Recall: 0.8999 - Accuracy: 0.9457 - F1: 0.9443
FIRE -> Precision: 0.9906 - Recall: 0.9484 - Accuracy: 0.9805 - F1: 0.9691
Mean F1 Score: 0.9567

________________________________ FASDD MEDIUM COMPRESSION ________________________________


Validating: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 65/65 [00:19<00:00,  3.32it/s]

SMOKE -> Precision: 0.9878 - Recall: 0.9154 - Accuracy: 0.9510 - F1: 0.9502
FIRE -> Precision: 0.9637 - Recall: 0.9731 - Accuracy: 0.9796 - F1: 0.9684
Mean F1 Score: 0.9593


### Only FASDD CV

In [19]:
print("__________________________________ FASDD NO COMPRESSION __________________________________")
_ = eval_onnx(fasdd_cv_loader, './models/onnx/fasdd_no_comp.onnx')
print("\n________________________________ FASDD MEDIUM COMPRESSION ________________________________")
_ = eval_onnx(fasdd_cv_loader, './models/onnx/fasdd_med.onnx')

__________________________________ FASDD NO COMPRESSION __________________________________


Validating: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 248/248 [00:48<00:00,  5.13it/s]


SMOKE -> Precision: 0.9021 - Recall: 0.8485 - Accuracy: 0.8887 - F1: 0.8744
FIRE -> Precision: 0.8560 - Recall: 0.9842 - Accuracy: 0.9378 - F1: 0.9156
Mean F1 Score: 0.8950

________________________________ FASDD MEDIUM COMPRESSION ________________________________


Validating: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 248/248 [00:44<00:00,  5.63it/s]

SMOKE -> Precision: 0.9066 - Recall: 0.8435 - Accuracy: 0.8888 - F1: 0.8739
FIRE -> Precision: 0.8824 - Recall: 0.9781 - Accuracy: 0.9478 - F1: 0.9278
Mean F1 Score: 0.9009
